In [9]:
import os
import sys
import numpy as np
import cPickle
import random
import math
import h5py
from multiprocessing import cpu_count
import scipy.io as sio
from sklearn.preprocessing import MultiLabelBinarizer
from collections import defaultdict
from gensim.models import Word2Vec, KeyedVectors
from six import iteritems
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle as skshuffle
from numpy import genfromtxt

from collections import defaultdict as dd
import tensorflow as tf
from tensorflow.python.ops.init_ops import glorot_uniform_initializer, RandomUniform, RandomNormal

from keras import layers
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec
from gensim.models.word2vec import Vocab

np.random.seed(1)
random.seed(1)

In [10]:

# GRADED FUNCTION: softmax

def softmax(x):
    """Calculates the softmax for each row of the input x.

    Your code should work for a row vector and also for matrices of shape (n, m).

    Argument:
    x -- A numpy matrix of shape (n,m)

    Returns:
    s -- A numpy matrix equal to the softmax of x, of shape (n,m)
    """
    
    ### START CODE HERE ### (≈ 3 lines of code)
    # Apply exp() element-wise to x. Use np.exp(...).
    x_exp = np.exp(x)

    # Create a vector x_sum that sums each row of x_exp. Use np.sum(..., axis = 1, keepdims = True).
    x_sum = np.sum(x_exp, axis = 1, keepdims = True)
    
    # Compute softmax(x) by dividing x_exp by x_sum. It should automatically use numpy broadcasting.
    s = x_exp / x_sum

    ### END CODE HERE ###
    
    return s

In [11]:
# Initialize parameters
DATASET = 'blogcatalog'

embedding_size = 128
learning_rate = 0.1
gl_learning_rate = 0.1
batch_size = 200
neg_samp = 0
model_file = 'trans.model'

window_size = 10
path_size = 10

g_batch_size = 200
g_learning_rate = 0.1
g_sample_size = 100

use_feature = True
update_emb = True
layer_loss =  True

In [4]:
# Load the dataset
NAMES = ['x', 'y', 'tx', 'ty', 'graph']
OBJECTS = []
for i in range(len(NAMES)):
    f = "/hdd2/graph_embedding/dataset/blogcatalog/trans.{}.{}".format(DATASET, NAMES[i])
    print(f)
    OBJECTS.append(cPickle.load(open(f)))
x, y, tx, ty, graph = tuple(OBJECTS)

/hdd2/graph_embedding/dataset/blogcatalog/trans.blogcatalog.x
/hdd2/graph_embedding/dataset/blogcatalog/trans.blogcatalog.y
/hdd2/graph_embedding/dataset/blogcatalog/trans.blogcatalog.tx
/hdd2/graph_embedding/dataset/blogcatalog/trans.blogcatalog.ty
/hdd2/graph_embedding/dataset/blogcatalog/trans.blogcatalog.graph


In [5]:
x.shape

(1031, 10312)

In [12]:
def comp_iter(iter):
    """an auxiliary function used for computing the number of iterations given the argument iter.
    iter can either be an int or a float.
    """
    if iter >= 1:
        return iter
    return 1 if random.random() < iter else 0

In [13]:
# Sample a collections of paths from the graph

In [14]:
def initialize_parameters(l_emd_f_W_size, l_x_hid_W_size, l_y_W_size):
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]??
                        W2 : [2, 2, 8, 16]?
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)
    
    l_emd_f_W = tf.Variable(tf.truncated_normal(l_emd_f_W_size,
                            stddev=1.0 / math.sqrt(l_emd_f_W_size[1])))
    l_emd_f_b = tf.Variable(tf.zeros([l_emd_f_W_size[0]]))
    
    l_x_hid_W = tf.get_variable('l_x_hid_W', shape = l_x_hid_W_size,
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_x_hid_b = tf.get_variable('l_x_hid_b', shape = [l_x_hid_W_size[0], 1],
                               initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_W = tf.get_variable('l_y_W', shape = l_y_W_size,
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    l_y_b = tf.get_variable('l_y_b', shape = [l_y_W_size[0], 1],
                           initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed = 1))
    
    parameters = {'l_emd_f_W': l_emd_f_W,
                  'l_emd_f_b': l_emd_f_b,
                  'l_x_hid_W': l_x_hid_W,
                  'l_x_hid_b': l_x_hid_b,
                  'l_y_W': l_y_W,
                  'l_y_b': l_y_b}
    
    return parameters

In [21]:
def create_placeholders():
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    
#     x_sym = tf.placeholder(tf.float32, shape = [None, x.shape[1]], name = 'x')
#     y_sym = tf.placeholder(tf.int32, shape = [None, y.shape[1]], name = 'y')
    g_sym = tf.placeholder(tf.int32, shape = [None, ], name = 'g')
    gy_sym = tf.placeholder(tf.int32, shape = [None, 1], name = 'gy')
    ind_sym = tf.placeholder(tf.int32, shape = [None], name = 'ind')
    
    path_sym = tf.placeholder(tf.int32, shape = [batch_size, path_size+1], name = 'path')
    path_id_sym = tf.placeholder(tf.int32, shape = [batch_size, ], name = 'path_id')
    w_path2pair_sym = tf.placeholder(tf.float32, shape = [batch_size, None], name = 'w_path2pair_sym')
    
    
    return g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym

In [16]:
file_list = ['/hdd2/graph_embedding/customized/blogcatalog.embeddings.walks.0']
dataset = genfromtxt(file_list[0], delimiter=' ')

def get_num_vacabulary(dataset):
    count = 0
    for d in dataset:
        count = max(count, max(d))
    return int(count)

In [29]:
np.unique(dataset)

array([  0.00000000e+00,   1.00000000e+00,   2.00000000e+00, ...,
         1.03090000e+04,   1.03100000e+04,   1.03110000e+04])

In [18]:
# num_sampled = 64
# num_ver = max(graph.keys()) + 1
# vocabulary_size = num_ver
vocabulary_size = get_num_vacabulary(dataset) + 1
n_hidden = 32
n_steps = path_size+1 #path length
n_input = 128 # embedding dim

In [19]:
window_size

10

In [26]:
# def build():
"""
Builds the model.
"""
np.random.seed(1)
random.seed(1)
tf.random_normal_initializer(seed = 1)

cgraph = tf.Graph()

with cgraph.as_default(), tf.device('/cpu:0'):

    g_sym, gy_sym, ind_sym, path_sym, path_id_sym, w_path2pair_sym = create_placeholders()

    # word embedding
    tf.random_normal_initializer(seed = 1)
    tf.set_random_seed(1)
    
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
#     softmax_weights = tf.Variable(
#         tf.random_normal([vocabulary_size, embedding_size], mean = 0.0, stddev = 0.01))
    softmax_weights = tf.Variable(
          tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
   
    l_emd_f = tf.nn.embedding_lookup(embeddings, g_sym)

    g_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=softmax_weights, biases=softmax_biases,
                                   inputs = l_emd_f, labels = gy_sym, 
                                   num_sampled = 5, 
                                   num_classes = vocabulary_size))    
    g_optimizer = tf.train.GradientDescentOptimizer(0.025).minimize(g_loss)
    
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm

In [23]:
def count_textfiles(file_list):
    count = -1
    num_path = 0
    for file_name in file_list:
        with open(file_name, 'r') as f:
            for line in f:
                num_path += 1
                for l in line.split():
                    count = max(count, int(l))
    return count, num_path

def get_word_pair(line):
    g = []
    list_path = []
    path = [int(t) for t in line.split()]
    list_path.append(path)
    for l in range(len(path)):
        for m in range(l - window_size, l + window_size + 1):
            if m < 0 or m >= len(path) or m == l: continue
            g.append([path[l], path[m]])
            gy.append(1.0)
    return g, list_path

def gen_graph_from_path_collection(data):
    g = []
    list_path = []
    
    for line in data:
        for l in xrange(len(line)):
#             reduced_window = np.random.randint(window_size)
            reduced_window = 0
            start = max(0, l - window_size + reduced_window)
            for m in range(start, l + window_size + 1 - reduced_window):
                if (m < 0 or m >= len(line) or m == l): 
                    continue
                g.append([int(line[l]), int(line[m])])
    return np.array(g, dtype = np.int32)

In [24]:
window_size = 10
representation_size = 128
embedding_filename = '/hdd2/graph_embedding/customized/blog_embeddings.txt'

In [27]:
# unsupervised word2vec
init_iter_deepwalk = 10
config = tf.ConfigProto(allow_soft_placement=True)
tot = 0
batch_path = 10000
with tf.Session(graph = cgraph, config=config) as session:
    tf.global_variables_initializer().run()
    
    for i in xrange(init_iter_deepwalk):
        tot_word = 0
        for batch_id in xrange(len(dataset) / batch_path + 1):
            start_idx = batch_path * batch_id
            end_idx = min(len(dataset), batch_path * (batch_id + 1))
            data_batch = dataset[start_idx: end_idx, :]
            gx = gen_graph_from_path_collection(data_batch)
            tot_word += gx.shape[0]
            print('start_idx: %d, end_idx: %d', %(start_idx, end_idx))
            print('processed words: %d' %gx.shape[0])
            feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
            _, l  = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
            
            print('iter: %d/%d, loss: %.4f' %(batch_id, len(dataset) / batch_path + 1, l))
    #             feed_dict={g_sym: np.array(range(0,v+1)), gy_sym: np.array(range(0,v+1)).reshape(v+1,1)}
    #             res_l_emd_f = session.run(l_emd_f, feed_dict=feed_dict)
#                 final_embeddings = normalized_embeddings.eval()
#                 embedding_filename_tmp = '/hdd2/graph_embedding/customized/blog_embeddings.%d.txt' %(tot)
    #             np.savetxt(embedding_filename_tmp, res_l_emd_f)
        tmp_embeddings = embeddings.eval()
        embedding_filename_tmp = '/hdd2/graph_embedding/customized/blog_embeddings.iter%d.txt' %(i)
        np.savetxt(embedding_filename_tmp, tmp_embeddings)
        
    
    final_embeddings = normalized_embeddings.eval()

('start_idx: %d, end_idx: %d', (0, 10000))
processed words: 6900000
iter: 0/83, loss: 21.8313
('start_idx: %d, end_idx: %d', (10000, 20000))
processed words: 6900000
iter: 1/83, loss: 19.7089
('start_idx: %d, end_idx: %d', (20000, 30000))
processed words: 6900000
iter: 2/83, loss: 25.2263
('start_idx: %d, end_idx: %d', (30000, 40000))
processed words: 6900000
iter: 3/83, loss: 27.6852
('start_idx: %d, end_idx: %d', (40000, 50000))
processed words: 6900000
iter: 4/83, loss: 35.6429
('start_idx: %d, end_idx: %d', (50000, 60000))
processed words: 6900000
iter: 5/83, loss: 38.0047
('start_idx: %d, end_idx: %d', (60000, 70000))
processed words: 6900000


KeyboardInterrupt: 

In [87]:
np.savetxt(embedding_filename, final_embeddings)

In [23]:
gx.shape

(11250000, 2)

iter: 0/83, loss: 1.0540


In [7]:
class Skipgram(Word2Vec):
    """A subclass to allow more customization of the Word2Vec internals."""
    
    def __init__(self, vocabulary_counts=None, **kwargs):

        self.vocabulary_counts = None

        kwargs["min_count"] = kwargs.get("min_count", 0)
        kwargs["workers"] = kwargs.get("workers", cpu_count())
        kwargs["size"] = kwargs.get("size", 128)
        kwargs["sentences"] = kwargs.get("sentences", None)
        kwargs["window"] = kwargs.get("window", 10)
        kwargs["sg"] = 1
        kwargs["hs"] = 1

        if vocabulary_counts != None:
            self.vocabulary_counts = vocabulary_counts

        super(Skipgram, self).__init__(**kwargs)
        
class WalksCorpus(object):
    def __init__(self, file_list):
        self.file_list = file_list
    def __iter__(self):
        for file in self.file_list:
            with open(file, 'r') as f:
                for line in f:
                    yield line.split()

In [26]:
import itertools
walks_corpus = WalksCorpus(file_list)

In [36]:
i = 0
for line in walks_corpus:
    i += 1
print(i)

824960


In [21]:
# model = Skipgram(sentences=walks_corpus, vocabulary_counts=vocabulary_size, size=128,
#                  window=10, min_count=0, trim_rule=None, workers=1)

In [31]:
1

1

In [88]:
def sparse2graph(x):
    G = defaultdict(lambda: set())
    cx = x.tocoo()
    for i,j,v in zip(cx.row, cx.col, cx.data):
        G[i].add(j)
    return {str(k): [str(x) for x in v] for k,v in iteritems(G)}

In [89]:
class TopKRanker(OneVsRestClassifier):
    def predict(self, X, top_k_list):
        assert X.shape[0] == len(top_k_list)
        probs = np.asarray(super(TopKRanker, self).predict_proba(X))
        all_labels = []
        for i, k in enumerate(top_k_list):
            probs_ = probs[i, :]
            labels = self.classes_[probs_.argsort()[-k:]].tolist()
            all_labels.append(labels)
        return all_labels

In [90]:
def scoring(emb_filename, matfile):
    # 0. Files
    embeddings_file = emb_filename

    # 1. Load Embeddings
    embeddings = np.loadtxt(embeddings_file)
    
    ## for original deepwalk
#     model = KeyedVectors.load_word2vec_format(embeddings_file, binary=False)

    # 2. Load labels
    mat = sio.loadmat(matfile)
    A = mat['network']
    graph = sparse2graph(A)
    labels_matrix = mat['group']
    labels_count = labels_matrix.shape[1]
    mlb = MultiLabelBinarizer(range(labels_count))

    # Map nodes to their features (note:  assumes nodes are labeled as integers 1:N)
    features_matrix = embeddings

#     features_matrix = np.asarray([model[str(node)] for node in range(len(graph))])
    
    # 2. Shuffle, to create train/test groups
    shuffles = []
    for x in range(1):
        shuffles.append(skshuffle(features_matrix, labels_matrix))

    # 3. to score each train/test group
    all_results = defaultdict(list)

#     if args.all:
#         training_percents = numpy.asarray(range(1, 10)) * .1
#     else:
#         training_percents = [0.1, 0.5, 0.9]
    training_percents = [0.1]
    for train_percent in training_percents:
        for shuf in shuffles:
            
            X, y = shuf

            training_size = int(train_percent * X.shape[0])

            X_train = X[:training_size, :]
            y_train_ = y[:training_size]

            y_train = [[] for x in range(y_train_.shape[0])]


            cy =  y_train_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_train[i].append(j)

            assert sum(len(l) for l in y_train) == y_train_.nnz

            X_test = X[training_size:, :]
            y_test_ = y[training_size:]

            y_test = [[] for _ in range(y_test_.shape[0])]

            cy =  y_test_.tocoo()
            for i, j in zip(cy.row, cy.col):
                y_test[i].append(j)

            clf = TopKRanker(LogisticRegression())
            clf.fit(X_train, y_train_)

            # find out how many labels should be predicted
            top_k_list = [len(l) for l in y_test]
            preds = clf.predict(X_test, top_k_list)

            results = {}
            averages = ["micro", "macro"]
            for average in averages:
                results[average] = f1_score(mlb.fit_transform(y_test), mlb.fit_transform(preds), average=average)

            all_results[train_percent].append(results)

    print ('Results, using embeddings of dimensionality', X.shape[1])
    print ('-------------------')
    for train_percent in sorted(all_results.keys()):
        print ('Train percent:', train_percent)
    for index, result in enumerate(all_results[train_percent]):
        print ('Shuffle #%d:   ' % (index + 1), result)
    avg_score = defaultdict(float)
    for score_dict in all_results[train_percent]:
        for metric, score in iteritems(score_dict):
            avg_score[metric] += score
    for metric in avg_score:
        avg_score[metric] /= len(all_results[train_percent])
    print ('Average score:', dict(avg_score))
    print ('-------------------')

In [91]:
matfile = '/hdd2/graph_embedding/deepwalk/example_graphs/blogcatalog.mat'
embedding_filename = '/hdd2/graph_embedding/customized/model_ns5_iter1.output'
scoring(embedding_filename, matfile)

('Results, using embeddings of dimensionality', 128)
-------------------
('Train percent:', 0.1)
('Shuffle #1:   ', {'micro': 0.16890388893150265, 'macro': 0.040154789900140916})
('Average score:', {'micro': 0.16890388893150265, 'macro': 0.040154789900140916})
-------------------


/home/mingzeng/anaconda2/lib/python2.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 38 is present in all training examples.
  str(classes[c]))
/home/mingzeng/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [108]:
mat = sio.loadmat(matfile)
A = mat['network']
labels_matrix = mat['group']
labels_count = labels_matrix.shape[1]
mlb = MultiLabelBinarizer(range(labels_count))

In [119]:
embeddings = np.loadtxt(embedding_filename)

In [120]:
embeddings.shape

(10311, 128)

In [121]:
labels_matrix.shape

(10312, 39)

In [87]:
init_iter_label = 10
init_iter_graph = 7
inst_generator = gen_train_inst()
graph_generator = gen_graph()
# Generates pairs with the same label (1) or different labels (-1)
label_generator = gen_label_graph()

In [17]:
max_iter = 1
iter_graph = 0
iter_inst = 5
iter_label = 0
use_reweight = False

In [92]:
# with tf.Session() as session:
#     gx, gy = next(label_generator)
#     tf.global_variables_initializer().run()
#     feed_dict={g_sym: gx[:, 0], gy_sym: gx[:, 1].reshape(gx.shape[0],1)}
#     _, loss = session.run([g_optimizer, g_loss], feed_dict=feed_dict)
#     print 'iter label', i, loss

In [93]:
res_reweight[0]

array([ 0.00497301], dtype=float32)

In [219]:
softmax((np.matmul(res_lstm_last_output, res_s_weight) + res_s_biase).reshape(1, -1))

array([[ 0.00496026,  0.00500135,  0.00504243,  0.0049207 ,  0.00508371,
         0.00487118,  0.00503232,  0.00513423,  0.00512543,  0.00511278,
         0.00495276,  0.00496932,  0.00509642,  0.00476472,  0.00508202,
         0.00513261,  0.00514402,  0.00488825,  0.00513993,  0.00504225,
         0.0049523 ,  0.00508264,  0.00510294,  0.0050783 ,  0.00510226,
         0.00496483,  0.00512717,  0.00503125,  0.00491823,  0.00499666,
         0.00503218,  0.00506654,  0.00481618,  0.00498946,  0.00489985,
         0.00475723,  0.00493511,  0.00506777,  0.00507806,  0.0049991 ,
         0.00499596,  0.00515256,  0.00484974,  0.0048238 ,  0.00505941,
         0.00483398,  0.00483752,  0.00492471,  0.00502468,  0.00476777,
         0.0050379 ,  0.00510353,  0.00502782,  0.00506198,  0.00492944,
         0.00501779,  0.00505026,  0.00508298,  0.00483345,  0.00514694,
         0.00506833,  0.00501812,  0.00487534,  0.00484263,  0.00506357,
         0.00490818,  0.00507065,  0.00500399,  0.0

In [215]:
res_lstm_last_output[0]

array([  2.98579922e-03,  -1.91092666e-03,   1.13703718e-03,
        -1.94945280e-03,  -0.00000000e+00,  -2.10431567e-03,
        -4.10453242e-04,   0.00000000e+00,   1.46083732e-03,
         0.00000000e+00,   1.93165115e-03,  -0.00000000e+00,
         3.66599881e-03,  -4.00959179e-05,  -6.41093543e-03,
        -2.04713945e-03,  -3.29010631e-03,   0.00000000e+00,
        -0.00000000e+00,   8.90581636e-04,   6.43362349e-04,
        -2.79952539e-03,   9.53049865e-04,   1.73583743e-04,
        -1.58014998e-03,  -4.64183697e-03,   2.87159951e-03,
        -0.00000000e+00,  -8.14037677e-03,  -1.79596210e-03,
         0.00000000e+00,   0.00000000e+00], dtype=float32)

In [353]:
# with tf.Session() as session:
# #     tf.global_variables_initializer().run()
#     feed_dict={x_sym: xs, y_sym: ys}
#     ll, l = session.run([optimizer, loss], feed_dict=feed_dict)

In [176]:
def iter_test():
    for i in xrange(2):
        yield i

In [185]:
test_generator = iter_test()

In [186]:
res_i = next(test_generator)
print(res_i)

0


Keys: [u'stream0']


[u'logits']

In [7]:
once = np.random.RandomState(hash('seed_string') & 0xffffffff)
len((once.rand(10322) - 0.5) / 10322)

10322